In [1]:
import datetime as dt

In [3]:
dates = [dt.datetime(2015, 1, 1), dt.datetime(2015, 7, 1),
        dt.datetime(2016, 1, 1)]

In [4]:
(dates[1] - dates[0]).days / 365

0.4958904109589041

In [5]:
(dates[2] - dates[1]).days / 365

0.5041095890410959

In [6]:
fractions = [0.0, 0.5, 1.0]

In [7]:
import numpy as np

In [10]:
def get_year_deltas(date_list, day_count=365.):
    """ Return vector of floats with day deltas in years.
    Initial value normalized to zero.
    
    
    Parameters
    ==========
    
    
    
    
    
    Results
    =======
    delta_list : array
        year fractions
    """
    
    
    start = date_list[0]
    delta_list = [(date - start).days / day_count for date in date_list]
    return np.array(delta_list)

In [11]:
get_year_deltas(dates)

array([ 0.        ,  0.49589041,  1.        ])

###  Constant short rate

In [ ]:
from get_year_deltas import *

In [12]:
class constant_short_rate(object):
    """ Class for constant short rate discounting.
    
    
    Parameters
    ==========
    
    
    
    
    
    Results
    =======
    """
    
    def __init__(self, name, short_rate):
        
        self.name = name
        self.short_rate = short_rate
        if short_rate < 0:
            raise ValueError('Short rate negative.')
            
            
    def get_discount_factors(self, date_list, dtobjects=True):
        
        if dtobjects is True:
            dlist = get_year_deltas(date_list)
        else:
            dlist = np.array(date_list)
        dflist = np.exp(self.short_rate * np.sort(-dlist))
        return np.array((date_list, dflist)).T
        
        
    
    
    
    

In [14]:
csr = constant_short_rate('csr', 0.05)

In [15]:
csr.get_discount_factors(dates)

array([[datetime.datetime(2015, 1, 1, 0, 0), 0.951229424500714],
       [datetime.datetime(2015, 7, 1, 0, 0), 0.9755103387657228],
       [datetime.datetime(2016, 1, 1, 0, 0), 1.0]], dtype=object)

In [16]:
deltas = get_year_deltas(dates)

In [17]:
csr.get_discount_factors(deltas, dtobjects=False)

array([[ 0.        ,  0.95122942],
       [ 0.49589041,  0.97551034],
       [ 1.        ,  1.        ]])

###  Market Environments

In [37]:
class market_environment(object):
    """ Class to model a market environment relevant for valuation
    
    
    Attributes
    ==========
    name: string
    
    
    
    Methods
    ==========
    add_constant :
    
    get_constant :
    
    add_list :
    
    get_list :
    
    add_curve :
    
    get_curve : 
    
    
    add_environment :
    
    
    """
    
    
    
    def __init__(self, name, pricing_date):
        self.name = name
        self.pricing_date = pricing_date
        self.constants = {}
        self.lists = {}
        self.curves = {}
        
        
    def add_constant(self, key, constant):
        
        self.constants[key] = constant
        
        
    def get_constant(self, key):
        return self.constant[key]
    
    def add_lists(self, key, list_object):
        self.lists[key] = list_object
        
    def get_list(self, key):
        return self.lists[key]
        
    def add_curve(self, key, curve):
        self.curves[key] = curve
    
    def get_curve(self, key):
        return self.curves[key]
        
    def add_environment(self, env):
        # overwrites existing values, if they exist
        for key in env.constants:
            self.constants[key] = env.constants[key]
        
        for key in env.lists:
            self.lists[key] = env.lists[key]
        
        for key in env.curves:
            self.curves[key] = env.curves[key]
        
        
        
        
    

In [38]:
dates = [dt.datetime(2015, 1, 1), dt.datetime(2015, 7, 1), dt.datetime(2016, 1, 1)]

In [39]:
csr = constant_short_rate('csr', 0.05)

In [40]:
me_1 = market_environment('me_1', dt.datetime(2015, 1, 1))

In [41]:
me_1.add_lists('symbols', ['AAPL', 'MSFT', 'FB'])

In [42]:
me_1.get_list('symbols')

['AAPL', 'MSFT', 'FB']

In [43]:
me_2 = market_environment('me_2', dt.datetime(2015, 1, 1))

In [44]:
me_2.add_constant('volatility', 0.2)

In [45]:
me_2.add_curve('short_rate', csr) # add instance of discounting class

In [46]:
me_2.get_curve('short_rate')

In [47]:
me_1.add_environment(me_2) # add complete environment

In [48]:
me_1.constants

{'volatility': 0.2}

In [50]:
me_1.lists

{'symbols': ['AAPL', 'MSFT', 'FB']}

In [51]:
me_1.curves

{'short_rate': <__main__.constant_short_rate at 0x7ffa701b25f8>}

In [52]:
me_1.get_curve('short_rate').short_rate

0.05